### Spark

In [1]:
# ссылка на документацию : https://spark.apache.org/docs/latest/api/python/reference/pyspark.html#rdd-apis
from pyspark.sql import SparkSession

In [29]:
# Локально создаем в Jupiter
spark = SparkSession\
    .builder\
    .master("local")\
    .appName("Python Spark SQL basic example")\
    .getOrCreate()
    # создаём объект Spark-сессии, обращаясь к объекту builder, который создаёт сессию, учитывая параметры конфигурации
# явно указываем, что хотим запустить Spark в локальном режиме
# задаём название нашего Spark-приложения
# функция инициализации объекта сессии


In [20]:
rdd = spark.sparkContext.parallelize([1,2,3,4,5,6,7])
print(rdd)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274


In [23]:
# Закрытие сессии
spark.stop()

### Создание коллекции RDD

💡 RDD(англ. resilient distributed dataset) — отказоустойчивый распределённый набор данных, или коллекция. Тип хранения данных, представляющий собой набор элементов, разделённых по узлам кластера, с которыми можно работать параллельно.


```python
 # Создание пустой коллекции Вариант 1
rdd2 = spark.sparkContext.emptyRDD()
 # Создание пустой коллекции Вариант 2
rdd2 = spark.sparkContext.parallelize([])
 # Создание коллекции из csv
rdd3 = spark.sparkContext.textFile('C:/tmp/files/text01.csv')


### Партиционирование

In [32]:
# посмотреть количество партиций
rdd = spark.sparkContext.parallelize([1, 2, 3, 4, 5, 6, 7])
rdd.getNumPartitions()

1

In [33]:
# repartition(val:int) Изменить количество партиций
rdd.repartition(5)
rdd.getNumPartitions()

1

In [34]:
# coalesce() Только уменьшает
rdd.coalesce(2)

CoalescedRDD[6] at coalesce at NativeMethodAccessorImpl.java:0

### Типы операций

In [36]:
rdd = spark.sparkContext.parallelize([1, 2, 3, 4, 5, 6, 7])
# map и в африке map
rdd2 = rdd.map(lambda x: x + 2)
rdd2.collect()

[3, 4, 5, 6, 7, 8, 9]

In [37]:
rdd = spark.sparkContext.parallelize('hello world')
# Может вернуть несколь объектов
rdd3 = rdd.flatMap(lambda x: x.split(','))
for i in rdd3.collect():
    print(i, end=' | ')

h | e | l | l | o |   | w | o | r | l | d | 

In [38]:
# Фильтрация
rdd = spark.sparkContext.parallelize([1,2,34,5,6,6,7,8,9,10])
rdd4 = rdd.filter(lambda x: x%2 == 0)
rdd4.collect()

[2, 34, 6, 6, 8, 10]

In [42]:
# Пересечение объектом
rdd = spark.sparkContext.parallelize(['hello','vs','vasya','world'])
rdd2 = spark.sparkContext.parallelize(['jopa','spark', 'hello','world'])
intersection = rdd.intersection(rdd2)
intersection.collect()

['world', 'hello']

In [47]:
# distinct() - уникальные значения
# union() - создание новой rdd из двух других
# sortByKey() - сортировка по ключу
# sortBy(function) - сортировка по ф-ции
# join()
# reduceByKey(func) - аналог groupby
# groupByKey() - возвращает интерируемый объект(похож на reduceByKey)
rdd = spark.sparkContext.parallelize(
    [
        ('beer',1),
        ('vodka',2),
        ('beer',5),
        ('milk',0),
        ('vodka',8)
    ]
)
rdd_reduce = rdd.reduceByKey(lambda x, y: x * y)
rdd_group = rdd.groupByKey()
print(f'{rdd_reduce.collect() = }')
print(f'{rdd_group.map(lambda x: (x[0], list(x[1]))).collect() = }')


rdd_reduce.collect() = [('beer', 5), ('vodka', 16), ('milk', 0)]
rdd_group.map(lambda x: (x[0], list(x[1]))).collect() = [('beer', [1, 5]), ('vodka', [2, 8]), ('milk', [0])]


In [49]:
spark.stop()

### Действия (actions)

|метод|что делает|последствия|
|--|--|--|
|`collect()`|Явно указывает собрать все части RDD|см.выше|
|`count()`|Считает количество элементов в дочерней RDD.|`count(rdd2)` -> 5|
|`take(n)`|Метод аналогичный `head()` в пандас|`rdd.take(2)`|
|`countByValue()`|Метод похож на reduceByKey(), но применяется, когда RDD состоит только из значений, которые нужно посчитать. Метод аналогичен value_counts()в библиотеке pandas.|`rdd.countByValue()`->`[(1,8),(2,3),(3,15)]`|
|`countByKey()`|Это действие аналогично методу reduceByKey(), так как суммирует по ключу. Отличие в том, что reduceByKey() — это трансформация, которая создаёт новую RDD, а countByKey() — это действие, которое запускает выполнение логического плана.||
|`reduce(function)`|явная инструкция на совершение агрегации. Он принимает агрегирующую функцию и применяет её ко всей RDD целиком.|`sc.parallelize([1, 2, 3, 4, 5]).reduce(add)` -> 15|
|``|||